## Handwritten digit recognition

#### Dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 
#### The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0,1,2,3,4,5,6,7,8,9), this is a classfication problem with 10 classes.
#### Our goal would be to build a neural network with 2 hidden layers.

# 1. Importing relevant packages.

In [2]:
import numpy as np

import tensorflow as tf
print(tf.__version__)

2.0.0-alpha0


In [9]:
import tensorflow_datasets as tfds

# 2. Data

In [10]:
mnist_dataset, mnist_info = tfds.load(name='mnist',with_info=True, as_supervised=True)

C:\Users\SrishtiKotnis\MyDocuments\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\SrishtiKotnis\MyDocuments\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\SrishtiKotnis\MyDocuments\lib\site-packages\urllib3\connectionpool.py:858: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
C:\Users\SrishtiKotnis\MyDocuments\lib\site-packages\urllib3\connectionpool.py:858

W0901 09:44:34.133733 18228 deprecation.py:323] From C:\Users\SrishtiKotnis\MyDocuments\lib\site-packages\tensorflow_datasets\core\file_format_adapter.py:209: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Dataset mnist downloaded and prepared to C:\Users\Srishti Kotnis\tensorflow_datasets\mnist\1.0.0. Subsequent calls will reuse this data.


W0901 09:44:58.784171 18228 dataset_builder.py:439] Warning: Setting shuffle_files=True because split=TRAIN and shuffle_files=None. This behavior will be deprecated on 2019-08-06, at which point shuffle_files=False will be the default for all splits.


In [25]:
mnist_train, mnist_test= mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1*mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = 0.1*mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image/=255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)
test_data = mnist_test.map(scale)


BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validatiion_samples)

BATCH_SIZE= 100

train_data =train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))


# 3. Model

## Outlining the model

In [60]:
input_size = 784
output_size = 10
hidden_layer_size = 100

model = tf.keras.Sequential([
                            tf.keras.layers.Flatten(input_shape=(28,28,1)),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                            tf.keras.layers.Dense(output_size,activation='softmax'),
                             ])

## Choosing the optimizer and the loss function 

In [61]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# 4. Training

In [62]:
num_epochs=5

model.fit(train_data, epochs = num_epochs, validation_data = (validation_inputs, validation_targets), verbose = 2)

Epoch 1/5
540/540 - 14s - loss: 0.3358 - accuracy: 0.9043 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
540/540 - 13s - loss: 0.1411 - accuracy: 0.9583 - val_loss: 0.1242 - val_accuracy: 0.9638
Epoch 3/5
540/540 - 13s - loss: 0.1019 - accuracy: 0.9697 - val_loss: 0.0942 - val_accuracy: 0.9730
Epoch 4/5
540/540 - 13s - loss: 0.0752 - accuracy: 0.9764 - val_loss: 0.0805 - val_accuracy: 0.9770
Epoch 5/5
540/540 - 13s - loss: 0.0601 - accuracy: 0.9816 - val_loss: 0.0651 - val_accuracy: 0.9805


# Testing the model

In [63]:
test_loss, test_accuracy = model.evaluate(test_data)

     10/Unknown - 0s 402ms/step - loss: 0.0882 - accuracy: 0.976 - 1s 299ms/step - loss: 0.0901 - accuracy: 0.976 - 1s 290ms/step - loss: 0.0924 - accuracy: 0.973 - 1s 285ms/step - loss: 0.0921 - accuracy: 0.973 - 1s 281ms/step - loss: 0.0919 - accuracy: 0.972 - 2s 280ms/step - loss: 0.0918 - accuracy: 0.972 - 2s 276ms/step - loss: 0.0910 - accuracy: 0.972 - 2s 278ms/step - loss: 0.0857 - accuracy: 0.973 - 2s 274ms/step - loss: 0.0848 - accuracy: 0.974 - 3s 273ms/step - loss: 0.0838 - accuracy: 0.974 - 3s 274ms/step - loss: 0.0838 - accuracy: 0.9745

In [64]:
print("Test loss: {0:.2f}. Test accuracy: {1:.2f}%".format(test_loss,test_accuracy*100.))

Test loss: 0.08. Test accuracy: 97.45%
